# Problema PyVista - Procesamiento de mayas

Este ejercicio va a consistir en la importación de una maya con `PyVista` y la aplicación de algunas transformaciones sencillas para familiarizarse con la librería.

> Realizado por Jorge Vila Tomás.

## Importación de las librerías

Para realizar el trabajo utilizaremos únicamente `NumPy` y `PyVista`:

In [1]:
import numpy as np
import pyvista as pv

## Carga de la malla

El primer paso que tendremos que realizar será cargar una malla que ya está creada. En este caso vamos a trabajar con una representación 3D de una arteria.

Para cargar la malla podemos utilizar la función `read`:

In [2]:
mesh = pv.read('Modelos_3D/CT_FFR_Pilot_6_corto.stl')
mesh

PolyData,Information
N Cells,34490
N Points,17247
X Bounds,"-1.699e+00, 5.406e+01"
Y Bounds,"-7.933e+01, -1.116e+01"
Z Bounds,"-1.690e+02, -1.466e+02"
N Arrays,0


Al mostrar el objeto que hemos cargado se pueden ver algunas características de la malla: cantidad de celdas y de puntos, límites, etc. También podemos acceder a esta información mediante los atributos correspondientes:

In [3]:
mesh.n_cells, mesh.n_points

(34490, 17247)

Por último, también podemos representar la malla en un visor 3D con el método `.plot()` de la misma:

In [4]:
mesh.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Modificando la malla: decimar y suavizar

`PyVista` permite modificar muy fácilmente las mallas. Para verlo aplicaremos las funciones de decimar y suavizar:

- Decimar: Consiste en reducir la cantidad de puntos de la malla. Lo podemos hacer con el método `.decimate()`. También podemos utilizar `.decimate_pro()`, que permite mantener la topología de la malla al hacer la operación.
- Suavizado: Se podría entender como la operación contraria. Aumenta la cantidad de puntos para suavizar los bordes. Podemos aplicarlo con el método `.smooth()`. *Nota: En la documentación no lo aplican directamente sobre la malla, si no que extraen la superficie con el método `.extract_geometry()` y aplican el suavizado a esta superficie.*

### Decimar

In [5]:
decimated = mesh.decimate(target_reduction=0.5)
decimated

PolyData,Information
N Cells,17244
N Points,8624
X Bounds,"-1.683e+00, 5.407e+01"
Y Bounds,"-7.933e+01, -1.117e+01"
Z Bounds,"-1.689e+02, -1.466e+02"
N Arrays,0


Si comparamos esta malla con la que hemos cargado anteriormente, comprobamos que tiene la mitad de puntos. Aún así, podemos representarla y ver que no se aprecia demasiado el cambio. Esta es una buena manera de reducir la carga computacional al poder trabajar con menos polígonos .

In [6]:
decimated.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Suavizar

In [7]:
surf = mesh.extract_geometry()
smooth = surf.smooth(n_iter=10)

En cambio, en este caso podemos ver que la cantidad de puntos de la malla se mantiene exactamente igual que en la malla original.

In [8]:
smooth

PolyData,Information
N Cells,34490
N Points,17247
X Bounds,"-1.694e+00, 5.403e+01"
Y Bounds,"-7.932e+01, -1.117e+01"
Z Bounds,"-1.690e+02, -1.466e+02"
N Arrays,0


Si representamos ahora esta malla, sí que podemos apreciar que se han suavizado los bordes:

In [9]:
smooth.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Cálculo de las normales

Podemos calcular las normales a cada cara utilizando el método `.compute_normals(inplace=True)`.

In [10]:
mesh.compute_normals(inplace=True)

PolyData (0x22ff10dc100)
  N Cells:	34490
  N Points:	17247
  X Bounds:	-1.699e+00, 5.406e+01
  Y Bounds:	-7.933e+01, -1.116e+01
  Z Bounds:	-1.690e+02, -1.466e+02
  N Arrays:	2

Y ahora las visualizamos:

> El parámetro `tolerance` indica la cantidad de flechas que vamos a mostrar. Un número más bajo implica una mayor cantidad de normales a enseñar.

In [11]:
arrows = mesh.glyph(scale="Normals", orient="Normals", tolerance=0.015)

In [12]:
p = pv.Plotter()
p.add_mesh(arrows, color="black")
p.add_mesh(mesh)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Campo escalar

`PyVista` también permite representar campos escalares junto con las mallas. Esto puede ser interesante si estamos trabajando con mallas que tienen asociados campos de presión, altura, temperatura, etc. A modo de ejemplo vamos a definir un campo contínuo de ejemplo que varía uniformemente para poder visualizarlo fácilmente.

El proceso es muy sencillo: simplemente tendremos que añadir el campo a la malla como si estuviésemos trabajando con un diccionario de Python, y aparecerá automáticamente en la representación 3D:

In [13]:
mesh['Sample Magnitude'] = np.arange(mesh.n_points)

In [14]:
p = pv.Plotter()
# p.add_mesh(arrows, color="black")
p.add_mesh(mesh)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Cálculo del volumen

Se supone que debería ser fácil siguiendo esto: https://docs.pyvista.org/examples/01-filter/compute-volume.html

In [15]:
mesh.compute_cell_sizes()

PolyData (0x22f84c05b80)
  N Cells:	34490
  N Points:	17247
  X Bounds:	-1.699e+00, 5.406e+01
  Y Bounds:	-7.933e+01, -1.116e+01
  Z Bounds:	-1.690e+02, -1.466e+02
  N Arrays:	6

## Análisis del interior de la malla: puntos interiores y exteriores

Otro ejercicio sencillo que podemos realizar consiste en rellenar el espacio con puntos aleatorios y dividirlos en dos conjuntos: los que están contenidos en la malla y los que están fuera. De esta manera podemos calcular incluso el volumen de la misma. Para hacerlo utilizaremos el método `.select_enclosed_points()`, que nos proporciona una máscara para diferenciar los puntos interiores y exteriores de la malla. 

Empezaremos generando los puntos aleatorios. Para facilitar el proceso los generamos únicamente en la caja que contiene a la malla:

In [16]:
xmin, xmax, ymin, ymax, zmin, zmax = mesh.bounds

In [17]:
N_points = 100000
a = np.random.uniform(low=xmin,
                      high=xmax,
                      size=(N_points,1))
b = np.random.uniform(low=ymin,
                      high=ymax,
                      size=(N_points,1))
c = np.random.uniform(low=zmin,
                      high=zmax,
                      size=(N_points,1))
d = np.concatenate([a,b,c], axis=-1)
d.shape

(100000, 3)

A partir de los puntos creados con `NumPy`, instanciamos un objeto `PolyData` de `PyVista`:

In [18]:
pts_pd = pv.PolyData(d)
pts_pd

PolyData,Information
N Cells,100000
N Points,100000
X Bounds,"-1.699e+00, 5.406e+01"
Y Bounds,"-7.933e+01, -1.116e+01"
Z Bounds,"-1.690e+02, -1.466e+02"
N Arrays,0


Si representamos los puntos junto con la malla anterior podemos comprobar que los hemos creado correctamente:

In [19]:
p = pv.Plotter()
p.add_mesh(pts_pd)
p.add_mesh(mesh)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Una vez tenemos las dos mallas (la original y la de puntos), solamente queda aplicar el método `malla1.select_enclosed_points(malla2)`. Hay que tener cuidado porque nos seleccionará los puntos de la `malla1` que están contenidos en la `malla2`, así que el orden es importante. En nuestro caso `malla1` será la nube de puntos y `malla2` la arteria:

In [20]:
pts_enclosed = pts_pd.select_enclosed_points(mesh)

La máscara que buscamos se encuentra en la clave `SelectedPoints`:

In [21]:
pts_enclosed['SelectedPoints']

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

A partir de ella es muy sencillo seleccionar los puntos interiores y los exteriores respecto de la malla:

In [22]:
pts_enc = pts_pd.extract_points(pts_enclosed['SelectedPoints'].view(bool))
pts_out = pts_pd.extract_points(~pts_enclosed['SelectedPoints'].view(bool))

Si los representamos de distinto color, vemos que el proceso se ha completado tal y como queríamos:

In [23]:
p = pv.Plotter()
p.add_points(pts_enc, color='r')
p.add_points(pts_out, color='b')
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Cálculo del volumen

Una vez hemos conseguido identificar la cantidad de puntos que están dentro de la malla, podemos calcular el volumen de la arteria como la proporción de puntos interiores multiplicado por el volumen total de la caja que hemos utilizado para generar los puntos:

In [24]:
lx, ly, lz = xmax-xmin, ymax-ymin, zmax-zmin

In [25]:
v = lx*ly*lz
v

85354.60418472321

In [26]:
v_art = (pts_enc.n_points/N_points)*v
v_art

1598.6917363798657

Finalmente obtenemos que la arteria tiene un volumen de 1599 unidades.